## **Evaluate Fine-Tuned Whisper**

### **GPU Setup**

In [1]:
import os
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 19 14:50:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              44W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Tell the progam to use the GPU allocated to us by setting the env variable used by CUDA
# Use the first GPU on your machine
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

**User Action Required**
- Select whisper version
- Select checkpoint number
- Select number of test examples

In [4]:
whisper_ver = 'whisper-base'

In [5]:
checkpoint_num = '2100'

In [6]:
num_test_examples = 600

### **GoogleDrive Environment Setup**

- Get stored model checkpoints

In [7]:
from google.colab import drive
google_drive_path = f'/content/drive/My Drive/{whisper_ver}-checkpoints'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Load Dataset**

In [8]:
!pip install datasets

In [9]:
from datasets import load_dataset
from IPython.display import Audio

In [10]:
dataset_repo_test = "johnlohjy/imda_nsc_p3_same_closemic_test"
dataset_test = load_dataset(dataset_repo_test, split='test', streaming=True, trust_remote_code=True)

### **Initialise Fine-tuned Whisper Model**

In [11]:
!pip install -q bitsandbytes accelerate

In [12]:
from transformers import WhisperForConditionalGeneration
from transformers import WhisperProcessor

In [13]:
checkpoint_path = f'/content/drive/My Drive/{whisper_ver}-checkpoints/checkpoint-{checkpoint_num}'

In [14]:
model = WhisperForConditionalGeneration.from_pretrained(checkpoint_path).to(device)
model.config.use_cache = True
processor = WhisperProcessor.from_pretrained(f"openai/{whisper_ver}", language="en", task="transcribe")

### **Prepare Dataset for Whisper**

In [15]:
def prepare_dataset(batch):
    # load audio data
    audio = batch["audio"]

    # Perform feature extraction: Compute log-Mel input features from input audio array
    # Use feature extractor to compute log-Mel spectrogram input features from 1D audio array
    # Pre-process raw audio-inputs
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # Perform tokenization: Encode target text to label ids
    # Encode transcriptions to label ids through use of tokenizer
    # Post-process model outputs to text format
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

In [16]:
dataset_test.column_names

['path', 'audio', 'sentence']

In [17]:
dataset_test_processed = dataset_test.map(prepare_dataset, remove_columns=['path','audio'])

### **Define Generation**

In [18]:
def transcribe(model, example):
    input_features = torch.tensor(example["input_features"]).unsqueeze(0).to(device)
    # Generate token IDs
    generated_ids = model.generate(input_features)
    # Decode token IDs to text
    predicted_transcription = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return predicted_transcription

### **Define Evaluation Metrics**

In [19]:
!pip install evaluate

In [20]:
!pip install jiwer

In [21]:
import evaluate

In [22]:
metric = evaluate.load("wer")

In [23]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
normalizer = BasicTextNormalizer()

def calculate_wer(model, dataset):
    predictions = []
    references = []

    for sample in dataset:
        transcription = transcribe(model, sample)
        reference = sample["sentence"]

        predictions.append(normalizer(transcription))
        references.append(normalizer(reference))


    # Compute WER between predictions and reference labels, as a percentage
    wer = 100 * metric.compute(predictions=predictions, references=references)

    return {"wer": wer}

### **Slice Test Set**

In [24]:
dataset_test_processed_iter_finetuned = iter(dataset_test_processed)

In [25]:
from itertools import islice
from torch.utils.data import IterableDataset

class SlicedDataset(IterableDataset):
    def __init__(self, dataset, num_examples):
        self.dataset = dataset
        self.num_examples = num_examples

    def __iter__(self):
        return islice(iter(self.dataset), self.num_examples)

    def __len__(self):
        return self.num_examples

dataset_test_processed_iter_finetuned_reduced = SlicedDataset(dataset_test_processed_iter_finetuned, num_examples=num_test_examples)

### **Evaluate the fine-tuned model**

In [26]:
calculate_wer(model, dataset_test_processed_iter_finetuned_reduced)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'wer': 12.840622853217312}